In [1]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA

# Load env
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

# Init LLM
llm = ChatOpenAI(
    model=OPENAI_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_BASE_URL,
    temperature=0,
)

# Build VectorStore
loader = TextLoader("../data/sample.txt")
docs = loader.load()

splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(split_docs, embeddings)
retriever = vectordb.as_retriever()

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

print("✅ Backend ready")


✅ Backend ready


In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import datetime

app = FastAPI()

class Query(BaseModel):
    message: str

# No RAG endpoint
@app.post("/chat/no_rag")
def api_no_rag(query: Query):
    response = llm.invoke(query.message)
    return {"answer": response.content}

# With RAG endpoint
@app.post("/chat/rag")
def api_rag(query: Query):
    response = rag_chain.invoke(query.message)
    answer = response["result"]
    sources = [doc.metadata.get("source", "sample.txt") for doc in response["source_documents"]]
    return {"answer": answer, "sources": list(set(sources))}

# Tool: get_current_time
@app.get("/tools")
def list_tools():
    return {
        "tools": [
            {
                "name": "get_current_time",
                "description": "Return the current server time",
                "input_schema": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        ]
    }

# No input needed, just return current time
@app.post("/call/get_current_time")
def get_current_time():
    now = datetime.datetime.now().isoformat()
    return {"answer": now}



In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [44365]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52602 - "POST /chat/no_rag HTTP/1.1" 200 OK
INFO:     127.0.0.1:52646 - "POST /chat/no_rag HTTP/1.1" 200 OK
INFO:     127.0.0.1:52725 - "POST /chat/no_rag HTTP/1.1" 200 OK
INFO:     127.0.0.1:52815 - "POST /chat/rag HTTP/1.1" 200 OK
INFO:     127.0.0.1:52868 - "POST /chat/rag HTTP/1.1" 200 OK
INFO:     127.0.0.1:52992 - "GET /tools HTTP/1.1" 200 OK
INFO:     127.0.0.1:52994 - "POST /call/chat_with_rag HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54740 - "GET /tools HTTP/1.1" 200 OK
INFO:     127.0.0.1:54742 - "POST /call/get_current_time HTTP/1.1" 200 OK
INFO:     127.0.0.1:54825 - "GET /tools HTTP/1.1" 200 OK
INFO:     127.0.0.1:54827 - "POST /call/chat_with_rag HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:55081 - "POST /chat/rag HTTP/1.1" 200 OK
INFO:     127.0.0.1:55306 - "GET /tools HTTP/1.1" 200 OK
INFO:     127.0.0.1:55310 - "POST /call/chat_with_rag HTTP/1.1" 404 Not Found
